In [1]:
# Add PATH
push!(LOAD_PATH, "/home/cfranken/code/gitHub/CLIMA_LSM/src/Leaf/");
push!(LOAD_PATH, "/home/cfranken/code/gitHub/CLIMA_LSM/src/Utils/");

In [2]:
using Leaf
using MathToolsMod
using LeafPhotosynthesisMod
using BenchmarkTools
using Plots

┌ Info: Recompiling stale cache file /home/cfranken/.julia/compiled/v1.1/MathToolsMod.ji for MathToolsMod [top-level]
└ @ Base loading.jl:1184
┌ Warning: Module MathToolsMod with build ID 9419434074488934 is missing from the cache.
│ This may mean MathToolsMod [top-level] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947


In [3]:
# Create a leaf structure
l = leaf_params{Float32}();
# Create a Flux structure
f = LeafPhotosynthesisMod.fluxes{Float32}();

In [4]:
# initialize some reasonable values
f.je = 100;
f.gbc = 100;
f.gbv = 100;
f.ceair=1500;
f.eair = 1500;
f.APAR = 500;

In [7]:
l.Kn = 2.44
LeafPhotosynthesisMod.LeafPhotosynthesis(f,l,298.0);

In [8]:
l

leaf_params{Float32}
  Kf: Float32 0.05f0
  Kd: Float32 0.85f0
  Kp: Float32 4.0f0
  Kn: Float32 2.44f0
  Kn_ss: Float32 1.6098189f0
  maxPSII: Float32 0.81632656f0
  effcon: Float32 0.2f0
  CO2_per_electron: Float32 0.12620832f0
  Ja: Float32 0.0f0
  C3: Bool true
  use_colim: Bool true
  gstyp: Int64 1
  o₂: Float32 209.0f0
  T: Float32 298.0f0
  esat: Float32 3140.9888f0
  desat: Float32 187.46216f0
  θ_j: Float32 0.9f0
  g0: Float32 0.01f0
  g1: Float32 9.0f0
  vpd_min: Float64 100.0
  gm: Float32 Inf32
  gs: Float32 0.16858746f0
  kc_25: Float32 404.9f0
  ko_25: Float32 278.4f0
  Γ_25_: Float32 42.75f0
  sco: Float32 2444.4443f0
  Γ_25: Float32 42.75f0
  kcha: Float32 79430.0f0
  koha: Float32 36380.0f0
  cpha: Float32 37830.0f0
  vcmaxha: Float32 65330.0f0
  jmaxha: Float32 43540.0f0
  rdha: Float32 46390.0f0
  vcmaxhd: Float32 150000.0f0
  jmaxhd: Float32 150000.0f0
  rdhd: Float32 150000.0f0
  vcmaxse: Float32 490.0f0
  jmaxse: Float32 490.0f0
  rdse: Float32 490.0f0
  vcmaxc: 

In [6]:
# Test speed 
@benchmark LeafPhotosynthesisMod.LeafPhotosynthesis(f,l,rand(Float32,1)[1]+300)

LoadError: UndefVarError: @benchmark not defined

In [7]:
@benchmark zbrent(f,l,LeafPhotosynthesisMod.CiFunc!,0.7*(rand(Float32,1)[1]+400),0.7*0.99*400,xtol=1.)

LoadError: UndefVarError: @benchmark not defined

In [8]:
@benchmark hybrid(f,l,LeafPhotosynthesisMod.CiFunc!,0.7*(rand(Float32,1)[1]+400),0.7*0.99*400,0.1)

LoadError: UndefVarError: @benchmark not defined

In [9]:
APAR = 0:100:1000

0:100:1000

In [10]:
l.vcmax25 = 90
l.jmax25 = 90*1.9
for f.APAR in APAR
    LeafPhotosynthesisMod.LeafPhotosynthesis(f,l,298.0);
    println(f.APAR, " ", f.ag, " ", f.an)
end

UndefVarError: UndefVarError: l not defined

In [11]:
@benchmark setLeafT!(l, rand(Float32,1)[1]+300)

LoadError: UndefVarError: @benchmark not defined